In [66]:
import pandas as pd
df = pd.read_csv('processd_data_0613.csv')

In [67]:
df['event_date']=pd.to_datetime(df['event_time']).dt.date
gr=df.groupby('user_id')

In [68]:
tmp_data = {'user_id' : df['user_id'].unique()}

In [69]:
tmp_data

{'user_id': array([1515915625519388267, 1515915625519380411, 1515915625513238515, ...,
        1515915625611024014, 1515915625611024020, 1515915625611024030])}

In [70]:
cur = df['event_date'].max()

data1 = gr.agg(
    user_last_event = ('event_date', lambda x: x.max()),                     # 유저가 마지막으로 발생시킨 이벤트의 시간
    user_session_count = ('user_session', lambda x: x.nunique()),            # 유저별 누적 세션 수
    total_visit = ('event_date', lambda x: x.nunique()),                     # 유저별 방문일 수
    view_count = ('event_type', lambda x : (x=='view').sum()),               # view 이벤트 수
    cart_count = ('event_type', lambda x : (x=='cart').sum()),               # 장바구니 이벤트 수
    purchase_count = ('event_type', lambda x : (x=='purchase').sum()),       # 구매 이벤트 수
    first_visit = ('event_date', lambda x : x.min()),                        # 첫 방문일
    last_first_visit = ('event_date', lambda x : (cur - x.min()).days)       # 첫 방문일로부터 지난 시간
)

# 첫 방문일로부터 지난 시간 파트 마지막 날짜의 애매 모호

In [90]:
data1.head()

,user_last_event,user_session_count,total_visit,view_count,cart_count,purchase_count,first_visit,last_first_visit
user_id,,,,,,,,
1515915625353226922,2020-10-29,1,1,1,0,0,2020-10-29,122
1515915625353230067,2020-10-06,1,1,1,0,0,2020-10-06,145
1515915625353230683,2020-12-12,4,3,13,0,0,2020-11-09,111
1515915625353230922,2020-10-02,1,1,1,0,0,2020-10-02,149
1515915625353234047,2021-02-18,1,10,36,0,0,2020-09-29,152


In [72]:
purchase_df = df[df['event_type'] == 'purchase']
purchase_gr = purchase_df.groupby('user_id')

data2 = purchase_gr.agg(
    total_price=('price', 'sum'),       # 유저별 총 소비액
    avg_price=('price', 'mean'),        # 유저별 평균 소비액
    min_price = ('price','min'),        # 유저별 min 소비액
    max_price = ('price','max')         # 유저별 max 소비액
)

In [91]:
data2.head()

,total_price,avg_price,min_price,max_price
user_id,,,,
1515915625353286099,119.03,119.030,119.03,119.03
1515915625353457259,55.16,55.160,55.16,55.16
1515915625353534622,57.15,19.050,19.05,19.05
1515915625353561691,345.72,172.860,172.86,172.86
1515915625353900095,57.85,28.925,18.10,39.75


In [94]:
# 세션 별 평균, 최대, 최소 시간
df['event_time'] = pd.to_datetime(df['event_time'])
session_gr = df.groupby(['user_id','user_session'])

session_df = session_gr['event_time'].agg(['min','max'])

session_df['session_duration'] = (session_df['max']- session_df['min']).dt.total_seconds()
session_df.head(10)

min  \
user_id             user_session                                                     
1515915625353226922 7qejzWzHlR                           2020-10-29 11:28:35+00:00   
1515915625353230067 ikPKHkuRhA                           2020-10-06 06:30:32+00:00   
1515915625353230683 5qXvZIBV2W                           2020-11-18 10:51:35+00:00   
                    AXbHvG5BRP                           2020-11-18 10:51:42+00:00   
                    YSbSzDlwcX                           2020-12-12 10:33:09+00:00   
                    dn9FkZ11dA                           2020-11-09 08:52:51+00:00   
1515915625353230922 PeKjQrbkxf                           2020-10-02 08:23:40+00:00   
1515915625353234047 2bc2f4b2-831e-44f4-bc0e-efabe4ca5253 2020-09-29 16:01:54+00:00   
1515915625353236157 jGh9mgZrOJ                           2021-02-04 04:19:46+00:00   
1515915625353238403 ba02c556-3722-419e-9105-905a93de780b 2020-10-16 09:49:22+00:00   

                                                                               max  \
user_id             user_session                                                     
1515915625353226922 7qejzWzHlR                           2020-10-29 11:28:35+00:00   
1515915625353230067 ikPKHkuRhA                           2020-10-06 06:30:32+00:00   
1515915625353230683 5qXvZIBV2W                           2020-11-18 10:59:26+00:00   
                    AXbHvG5BRP                           2020-11-18 10:51:42+00:00   
                    YSbSzDlwcX                           2020-12-12 10:33:09+00:00   
                    dn9FkZ11dA                           2020-11-09 09:26:38+00:00   
1515915625353230922 PeKjQrbkxf                           2020-10-02 08:23:40+00:00   
1515915625353234047 2bc2f4b2-831e-44f4-bc0e-efabe4ca5253 2021-02-18 09:11:44+00:00   
1515915625353236157 jGh9mgZrOJ                           2021-02-11 16:10:01+00:00   
1515915625353238403 ba02c556-3722-419e-9105-905a93de780b 2020-10-16 09:49:22+00:00   

                                                          session_duration  
user_id             user_session                                            
1515915625353226922 7qejzWzHlR                                         0.0  
1515915625353230067 ikPKHkuRhA                                         0.0  
1515915625353230683 5qXvZIBV2W                                       471.0  
                    AXbHvG5BRP                                         0.0  
                    YSbSzDlwcX                                         0.0  
                    dn9FkZ11dA                                      2027.0  
1515915625353230922 PeKjQrbkxf                                         0.0  
1515915625353234047 2bc2f4b2-831e-44f4-bc0e-efabe4ca5253        12244190.0  
1515915625353236157 jGh9mgZrOJ                                    647415.0  
1515915625353238403 ba02c556-3722-419e-9105-905a93de780b               0.0

In [75]:
data3 = session_df.groupby('user_id').agg(
    avg_session_time = ('session_duration', lambda x : x.mean().round(2)),
    total_session_time = ('session_duration', lambda x : x.sum())
)

In [92]:
data3.head()

,avg_session_time,total_session_time
user_id,,
1515915625353226922,0.0,0.0
1515915625353230067,0.0,0.0
1515915625353230683,624.5,2498.0
1515915625353230922,0.0,0.0
1515915625353234047,12244190.0,12244190.0


In [79]:
# 함수 만들기 brand
def function_brand_find2(find_df):
    result = []
    for event_type in ['purchase', 'view', 'cart']:
        
        event_df = find_df[find_df['event_type'] == event_type]
        user_group = event_df.groupby(['user_id', 'brand']).size().reset_index(name='count')
        top_brand = user_group.loc[user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)
        top_brand = top_brand[['user_id', 'brand']].rename(columns={'brand': f'top_{event_type}_brand'})
        
        
        result.append(top_brand)
        
    merged_result = result[0]
    for result in result[1:]:
        merged_result = pd.merge(merged_result, result, on='user_id', how='outer')
    return merged_result

user_top_brands2 = function_brand_find2(df)

In [80]:
user_top_brands2

,user_id,top_purchase_brand,top_view_brand,top_cart_brand
0,1515915625353226922,NaN,honor,NaN
1,1515915625353230067,NaN,kester,NaN
2,1515915625353230683,NaN,creative,NaN
3,1515915625353230922,NaN,msi,NaN
4,1515915625353234047,NaN,samsung,NaN
...,...,...,...,...
407278,1515915625611023671,NaN,others,NaN
407279,1515915625611023730,NaN,others,NaN
407280,1515915625611024014,NaN,starwind,NaN
407281,1515915625611024020,NaN,amazfit,NaN


In [82]:
# 함수 만들기 category
def function_category_find2(find_df):
    result = []
    for event_type in ['purchase', 'view', 'cart']:
        
        event_df = find_df[find_df['event_type'] == event_type]
        user_group = event_df.groupby(['user_id', 'main_category']).size().reset_index(name='count')
        top_category = user_group.loc[user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)
        top_category = top_category[['user_id', 'main_category']].rename(columns={'main_category': f'top_{event_type}_category'})
        
        
        result.append(top_category)
        
    merged_result = result[0]
    for result in result[1:]:
        merged_result = pd.merge(merged_result, result, on='user_id', how='outer')
    return merged_result

user_top_category2 = function_category_find2(df)

In [83]:
user_top_category2

,user_id,top_purchase_category,top_view_category,top_cart_category
0,1515915625353226922,NaN,electronics,NaN
1,1515915625353230067,NaN,others,NaN
2,1515915625353230683,NaN,electronics,NaN
3,1515915625353230922,NaN,computers,NaN
4,1515915625353234047,NaN,electronics,NaN
...,...,...,...,...
407278,1515915625611023671,NaN,electronics,NaN
407279,1515915625611023730,NaN,others,NaN
407280,1515915625611024014,NaN,electronics,NaN
407281,1515915625611024020,NaN,electronics,NaN


In [84]:
user_data = data1.merge(data2, on='user_id', how='outer')
user_data = user_data.merge(data3, on='user_id', how='outer')
user_data = user_data.merge(user_top_brands2, on='user_id', how='outer')
user_data = user_data.merge(user_top_category2, on='user_id', how='outer')

In [95]:
# 유니크 user_id DataFrame
tmp_data = pd.DataFrame(tmp_data, columns=['user_id'])
tmp_data.head(2)

,user_id
0,1515915625519388267
1,1515915625519380411


In [87]:
merge_final = tmp_data.merge(user_data, on='user_id', how='left')

In [88]:
merge_final.info()
merge_final.to_csv('merge_final.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407283 entries, 0 to 407282
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                407283 non-null  int64  
 1   user_last_event        407283 non-null  object 
 2   user_session_count     407283 non-null  int64  
 3   total_visit            407283 non-null  int64  
 4   view_count             407283 non-null  int64  
 5   cart_count             407283 non-null  int64  
 6   purchase_count         407283 non-null  int64  
 7   first_visit            407283 non-null  object 
 8   last_first_visit       407283 non-null  int64  
 9   total_price            21304 non-null   float64
 10  avg_price              21304 non-null   float64
 11  min_price              21304 non-null   float64
 12  max_price              21304 non-null   float64
 13  avg_session_time       407283 non-null  float64
 14  total_session_time     407283 non-nu

code 하나하나 짜보기

In [ ]:
# 유저가 마지막으로 발생시킨 이벤트의 시간
gr.agg(
    user_last_event = ('event_date', lambda x: x.max())
)

In [ ]:
# 유저별 누적 세션수
gr.agg(
    user_session_count = ('user_session', lambda x: x.nunique())
)

In [ ]:
# 유저별 누적 세션수
gr.agg(
    user_session_count = ('user_session', lambda x: x.nunique())
)

In [ ]:
# 이벤트별 누적 수

# df['event_type'].value_counts()

# # 유저별 view 누적 수
# gr.agg(view_count = ('event_type', lambda x : (x=='view').sum()))
# # 유저별 cart 누적 수
# gr.agg(cart_count = ('event_type', lambda x : (x=='cart').sum()))
# # 유저별 purchase 누적 수
# gr.agg(purchase_count = ('event_type', lambda x : (x=='purchase').sum()))
gr.agg(
    view_count = ('event_type', lambda x : (x=='view').sum()),
    cart_count = ('event_type', lambda x : (x=='cart').sum()),
    purchase_count = ('event_type', lambda x : (x=='purchase').sum()),
)

---
===

In [ ]:
# 유저별 누적 소비액
gr.agg(total_price = ('event_type', lambda x : df.loc[x.index, 'price'][x=='purchase'].sum()))

In [ ]:
# purchase 다 건들이려 하니 연산이 너무 오래 걸림
purchase_df = df[df['event_type'] == 'purchase']
purchase_gr = purchase_df.groupby('user_id')

In [ ]:
# 그룹화한 pruchase data group 사용
purchase_gr.agg(total_price=('price', 'sum'))

In [ ]:
# 유저별 평균 소비액
purchase_gr.agg(avg_price=('price', 'mean'))

In [ ]:
purchase_df = df[df['event_type'] == 'purchase']
purchase_gr = purchase_df.groupby('user_id')

purchase_gr.agg(
    total_price=('price', 'sum'),
    avg_price=('price', 'mean')
)

---
===

In [ ]:
# 유저별 min &max 소비액
purchase_gr.agg(
    min_price = ('price','min'),
    max_price = ('price','max')
)

In [ ]:
# 유저별 첫 방문
gr.agg(
    first_visit = ('event_date', lambda x : x.min())
)

In [ ]:
# 유저 첫 방문일로부터 지난 시간

# 기준이 이 데이터의 마지막 날이라면
cur = df['event_date'].max()
# 기준이 오늘이라면
# cur = pd.to_datetime('now')
gr.agg(
    last_first_visit = ('event_date', lambda x : (cur - x.min()).days)
)

# 기준이 유저의 마지막 접속 날짜라면
# gr.agg(
#     last_first_visit = ('event_date', lambda x : (x.max() - x.min()).days)
# )

In [ ]:
# 체류 시간 ( 규림님 코드 )
session_df = session_gr['event_time'].agg(['min','max'])

session_df['session_duration'] = (session_df['max']- session_df['min']).dt.total_seconds()
session_df.head(10)

In [ ]:
session_df.groupby('user_id').agg(
    avg_session_time = ('session_duration', lambda x : x.mean().round(2)),
    total_session_time = ('session_duration', lambda x : x.sum())
)

---
===

In [12]:
# 'purchase' 이벤트만 필터링
purchase_df = df[df['event_type'] == 'purchase']

# 유저별로 그룹화하여 가장 많이 본 브랜드 계산 함수
def most_frequent_brand(brands):
    try:
        top_brand = brands.value_counts().idxmax()
    except ValueError:  # 빈 시퀀스 처리
        top_brand = None
    return top_brand

# 유저별로 그룹화하여 함수 적용
user_top_perchase_brand = purchase_df.groupby('user_id').agg(
    top_perchase_brand=('brand', lambda x: most_frequent_brand(x))
).reset_index()

# 결과 확인
user_top_perchase_brand

,user_id,top_perchase_brand
0,1515915625353286099,zyxel
1,1515915625353457259,asus
2,1515915625353534622,samsung
3,1515915625353561691,keenetic
4,1515915625353900095,sven
...,...,...
21299,1515915625610995356,None
21300,1515915625610997879,msi
21301,1515915625610999486,gigabyte
21302,1515915625611008742,None


In [13]:
# 'view' 이벤트만 필터링
view_df = df[df['event_type'] == 'view']

# 유저별로 그룹화하여 가장 많이 본 브랜드 계산 함수
def most_frequent_brand(brands):
    try:
        top_brand = brands.value_counts().idxmax()
    except ValueError:  # 빈 시퀀스 처리
        top_brand = None
    return top_brand

# 유저별로 그룹화하여 함수 적용
user_top_view_brand = view_df.groupby('user_id').agg(
    top_view_brand=('brand', lambda x: most_frequent_brand(x))
).reset_index()

# 결과 확인
user_top_view_brand

,user_id,top_view_brand
0,1515915625353226922,honor
1,1515915625353230067,kester
2,1515915625353230683,creative
3,1515915625353230922,msi
4,1515915625353234047,samsung
...,...,...
406858,1515915625611023671,None
406859,1515915625611023730,None
406860,1515915625611024014,starwind
406861,1515915625611024020,amazfit


In [14]:
# 'cart' 이벤트만 필터링
cart_df = df[df['event_type'] == 'cart']

# 유저별로 그룹화하여 가장 많이 본 브랜드 계산 함수
def most_frequent_brand(brands):
    try:
        top_brand = brands.value_counts().idxmax()
    except ValueError:  # 빈 시퀀스 처리
        top_brand = None
    return top_brand

# 유저별로 그룹화하여 함수 적용
user_top_cart_brand = cart_df.groupby('user_id').agg(
    top_cart_brand=('brand', lambda x: most_frequent_brand(x))
).reset_index()

# 결과 확인
user_top_cart_brand

,user_id,top_cart_brand
0,1515915625353286099,zyxel
1,1515915625353457259,asus
2,1515915625353534622,samsung
3,1515915625353561691,keenetic
4,1515915625353706476,philips
...,...,...
36947,1515915625611011782,None
36948,1515915625611013140,gigabyte
36949,1515915625611015921,msi
36950,1515915625611017481,sapphire


In [15]:
user_top_brands = user_top_view_brand.merge(user_top_cart_brand, on='user_id', how='outer')
user_top_brands = user_top_brands.merge(user_top_perchase_brand, on='user_id', how='outer')

In [16]:
user_top_brands

,user_id,top_view_brand,top_cart_brand,top_perchase_brand
0,1515915625353226922,honor,NaN,NaN
1,1515915625353230067,kester,NaN,NaN
2,1515915625353230683,creative,NaN,NaN
3,1515915625353230922,msi,NaN,NaN
4,1515915625353234047,samsung,NaN,NaN
...,...,...,...,...
407278,1515915625611023671,None,NaN,NaN
407279,1515915625611023730,None,NaN,NaN
407280,1515915625611024014,starwind,NaN,NaN
407281,1515915625611024020,amazfit,NaN,NaN


In [17]:
# 함수 만들기
def function_brand_find(find_df):
    result = {}
    for event_type in ['purchase', 'view', 'cart']:
        event_df = find_df[find_df['event_type'] == event_type]
        if not event_df.empty:
            try:
                top_brand = event_df['brand'].value_counts().idxmax()
            except ValueError:
                top_brand = None
        else:
            top_brand = None
        result[f'top_{event_type}_brand'] = top_brand
    return pd.Series(result)

In [18]:
# 유저별 가장 많이 purchase, view, cart 한 브랜드
user_top_brands = gr.apply(function_brand_find).reset_index()

/var/folders/kw/s1p33ssd2q94nb1csdw73sdw0000gn/T/ipykernel_5192/3945287129.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_top_brands = gr.apply(function_brand_find).reset_index()


In [19]:
user_top_brands

,user_id,top_purchase_brand,top_view_brand,top_cart_brand
0,1515915625353226922,None,honor,None
1,1515915625353230067,None,kester,None
2,1515915625353230683,None,creative,None
3,1515915625353230922,None,msi,None
4,1515915625353234047,None,samsung,None
...,...,...,...,...
407278,1515915625611023671,None,None,None
407279,1515915625611023730,None,None,None
407280,1515915625611024014,None,starwind,None
407281,1515915625611024020,None,amazfit,None


In [24]:
user_top_brands[user_top_brands['user_id']==1515915625353900095]

,user_id,top_purchase_brand,top_view_brand,top_cart_brand
54,1515915625353900095,sven,canyon,canyon


In [23]:
tmp = df[(df['user_id']==1515915625353900095)][['user_id', 'event_type', 'brand']]
tmp.groupby(['user_id', 'event_type']).value_counts()

user_id              event_type  brand  
1515915625353900095  cart        canyon      1
                                 sven        1
                     purchase    canyon      1
                                 sven        1
                     view        canyon     15
                                 sven       10
                                 optoma      5
                                 amd         1
                                 bbk         1
                                 cactus      1
                                 pantum      1
                                 philips     1
                                 ricoh       1
Name: count, dtype: int64

고객의 event type 별 최빈값 

In [ ]:
tmp_df = df

In [ ]:
# 가설 : 그룹을 두개를 잡고 갯수를 세면 더 빨라지지 않을까?

# event_type 이 view인 데이터만 테스트
view_df = tmp_df[tmp_df['event_type'] == 'view']
# 유저 별 브랜드 와 그 수
view_user_group = view_df.groupby(['user_id', 'brand']).size().reset_index(name='count')

In [ ]:
view_top_brand = view_user_group.loc[view_user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)

In [ ]:
view_df = tmp_df[tmp_df['event_type'] == 'view']
purchase_df = tmp_df[tmp_df['event_type'] == 'purchase']
cart_df = tmp_df[tmp_df['event_type'] == 'cart']


view_user_group = view_df.groupby(['user_id', 'brand']).size().reset_index(name='count')
purchase_user_group = purchase_df.groupby(['user_id', 'brand']).size().reset_index(name='count')
cart_user_group = cart_df.groupby(['user_id', 'brand']).size().reset_index(name='count')

view_top_brand = view_user_group.loc[view_user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)
purchase_top_brand = purchase_user_group.loc[purchase_user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)
cart_top_brand = cart_user_group.loc[cart_user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)

In [ ]:
view_top_brand = view_top_brand[['user_id', 'brand']].rename(columns={'brand': 'top_view_brand'})
purchase_top_brand = purchase_top_brand[['user_id', 'brand']].rename(columns={'brand': 'top_purchase_brand'})
cart_top_brand = cart_top_brand[['user_id', 'brand']].rename(columns={'brand': 'top_cart_brand'})

In [ ]:
merge_df = purchase_top_brand.merge(view_top_brand, how = 'outer')
merge_df = merge_df.merge(cart_top_brand, how = 'outer')

In [ ]:
merge_df

In [ ]:
# 함수 만들기 brand
def function_brand_find2(find_df):
    result = []
    for event_type in ['purchase', 'view', 'cart']:
        
        event_df = find_df[find_df['event_type'] == event_type]
        user_group = event_df.groupby(['user_id', 'brand']).size().reset_index(name='count')
        top_brand = user_group.loc[user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)
        top_brand = top_brand[['user_id', 'brand']].rename(columns={'brand': f'top_{event_type}_brand'})
        
        
        result.append(top_brand)
        
    merged_result = result[0]
    for result in result[1:]:
        merged_result = pd.merge(merged_result, result, on='user_id', how='outer')
    return merged_result

user_top_brands2 = function_brand_find2(df)

In [ ]:
# 함수 만들기 category
def function_category_find2(find_df):
    result = []
    for event_type in ['purchase', 'view', 'cart']:
        
        event_df = find_df[find_df['event_type'] == event_type]
        user_group = event_df.groupby(['user_id', 'main_category']).size().reset_index(name='count')
        top_category = user_group.loc[user_group.groupby('user_id')['count'].idxmax()].reset_index(drop=True)
        top_category = top_category[['user_id', 'main_category']].rename(columns={'main_category': f'top_{event_type}_category'})
        
        
        result.append(top_category)
        
    merged_result = result[0]
    for result in result[1:]:
        merged_result = pd.merge(merged_result, result, on='user_id', how='outer')
    return merged_result

user_top_category2 = function_category_find2(df)

쓰레기통
===

In [11]:
# 함수 만들기
# def function_brand_find(x, event_type):
    # tmp = x[df.loc[x.index, 'event_type'] == event_type]
    # tmp_df = df[df['event_type'] == event_type]
    # try:
    #     tmp_gr = tmp_df.groupby('user_id')['brand'].apply(lambda x: x.value_counts().idxmax())
    #     return tmp_gr.get(x.name, None)
    # except:
    #     return None
    # if tmp.empty:
    #     return None
    # # if len(tmp) == 0:
    # #     return None
    # # elif len(tmp) == 1:
    # #     t1 = tmp.value_counts()
    # #     return t1[0]
    # else:
    #     try:
    #         result = tmp.value_counts().idxmax(skipna=False)
    #         return result
    #     except:
    #         return None
    #     return None
        
    #     return tmp.value_counts()
    
# 테스트 결과 실패

In [ ]:
# 유저별 가장 많이 purchase, view, cart 한 브랜드
# gr=df.groupby('user_id')
# gr.agg(
#     top_purchase_brand = ('brand', lambda x: x[df.loc[x.index, 'event_type'] == 'purchase'].value_counts().idxmax() if not x[df.loc[x.index, 'event_type'] == 'purchase'].empty else None)
# )

# 빈 값 처리 불가능 폐기

In [ ]:
# purchase_df = df[df['event_type'] == 'purchase']
# view_df = df[df['event_type'] == 'view']
# cart_df = df[df['event_type'] == 'cart']
# 
# def find_top_brands(user_df):
#     result = {}
#     if not purchase_df.empty:
#         
#         try:
#             top_brand = purchase_df['brand'].value_counts().idxmax()
#         except:
#             top_brand = None
#     else:
#         top_brand = None
#     result['top_purchase_brand'] = top_brand
#     
#     
    
    # for event_type in ['purchase', 'view', 'cart']:
    #     filtered_df = user_df[user_df['event_type'] == event_type]
    #     if not filtered_df.empty:
    #         try:
    #             top_brand = filtered_df['brand'].value_counts().idxmax()
    #         except:
    #             top_brand = None
    #     else:
    #         top_brand = None
    #     result[f'top_{event_type}_brand'] = top_brand
    # return pd.Series(result)
    
# user_top_brands = df.groupby('user_id').apply(find_top_brands).reset_index()

In [ ]:
# 유저별 가장 많이 구매한 브랜드를 찾는 함수
# def find_top_brands(user_df):
#     result = {}
#     
#     purchase_df = user_df[user_df['event_type'] == 'purchase']
#     view_df = df[df['event_type'] == 'view']
#     cart_df = df[df['event_type'] == 'cart']
#     
#     if not purchase_df.empty:
#         try:
#             top_brand = purchase_df['brand'].value_counts().idxmax()
#         except:
#             top_brand = None
#     else:
#         top_brand = None
#     
#     result['top_purchase_brand'] = top_brand
#     
#     if not view_df.empty:
#         try:
#             top_brand = view_df['brand'].value_counts().idxmax()
#         except:
#             top_brand = None
#     else:
#         top_brand = None
#     
#     result['top_view_brand'] = top_brand
#     
#     if not cart_df.empty:
#         try:
#             top_brand = cart_df['brand'].value_counts().idxmax()
#         except:
#             top_brand = None
#     else:
#         top_brand = None
#     
#     result['top_cart_brand'] = top_brand
#     
#     return pd.Series(result)
# 
# # 유저별로 그룹화하여 함수 적용
# user_top_brands = df.groupby('user_id').apply(find_top_brands).reset_index()

In [ ]:
# user_top_brands